## 1. Load the dataset



To run this notebook, you will need to install: pandas, openai, transformers, plotly, matplotlib, scikit-learn, torch (transformer dep), torchvision, and scipy.

In [2]:
# imports
import openai
import pandas as pd
import tiktoken
from openai.embeddings_utils import get_embedding

# Set API_KEY
openai.api_key = "sk-"


ModuleNotFoundError: No module named 'tiktoken'

In [3]:
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191


In [23]:
# load & inspect dataset
input_datapath = pd.read_sql_query("SELECT * FROM 'mortgage complaints'", "sqlite:///StaterData.db")

#Limit test size due to performance issues
data = input_datapath.loc[:100]

df = data["Consumer complaint narrative"]

df.head(3)

0    My father passed away on XXXX and mother passe...
1    Dear XXXX, During Historical event of XXXX XXX...
2    The problem began approx. XX/XX/2022. I had a ...
Name: Consumer complaint narrative, dtype: object

## 2. Get embeddings and save them for future reuse

In [1]:
# top_n = 1000
# df = df.tail(top_n * 2)  # first cut to first 2k entries, assuming less than half will be filtered out

encoding = tiktoken.get_encoding(embedding_encoding)

# omit reviews that are too long to embed
df["n_tokens"] = df.apply(lambda x: len(encoding.encode(str(x))))
df = df[df["n_tokens"] <= max_tokens].tail(top_n)
len(df)

df.head(3)

NameError: name 'tiktoken' is not defined

In [26]:
# Apply and save the CSV (Uses OPENAI tokens)
df.apply(lambda x: get_embedding(x, engine=embedding_model))
df.to_csv("MortgageEmbeddings.csv")

RetryError: RetryError[<Future at 0x2e4493b1b80 state=finished raised TypeError>]